In [7]:
from tensorflow import keras
from augmentation import *
from keras.utils import to_categorical
from keras.layers import GlobalAvgPool2D, MaxPool2D, Dense, Input
from keras.optimizers import SGD
from imgaug import augmenters as iaa
from architectures import composite_convolutional_layer
import matplotlib.pyplot as plt
import tensorflow as tf


In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Convert labels to one-hot encoding
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [19]:
# Define CNN architecture
learning_rate = 0.01
momentum = 0.09
salt_pepper_amount = 0.06
batch_size = 64
dropout_rate = 0.2
epochs = 50

model = keras.Sequential(
    [
        Input(shape=(32, 32, 3)),

        *composite_convolutional_layer(32, 2, dropout_rate=dropout_rate),
        MaxPool2D((2, 2)),

        *composite_convolutional_layer(64, 2, dropout_rate=dropout_rate),
        MaxPool2D((2, 2)),

        *composite_convolutional_layer(64, 2, dropout_rate=dropout_rate),
        MaxPool2D((2, 2)),

        *composite_convolutional_layer(64, 2, dropout_rate=dropout_rate),
        MaxPool2D((2, 2)),

        GlobalAvgPool2D(),
        Dense(num_classes, activation="softmax"),
    ]
)


In [20]:
# Compile model
model.compile(optimizer=SGD(learning_rate, momentum), loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 32, 32, 32)        416       
                                                                 
 batch_normalization_9 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_9 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 dropout_9 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 16, 16, 64)       

In [21]:
# Train model
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

# Evaluate model on test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)

Epoch 1/50
782/782 [==============================] - 65s 82ms/step - loss: 1.8601 - accuracy: 0.3251 - val_loss: 1.9172 - val_accuracy: 0.2478
Epoch 2/50
782/782 [==============================] - 61s 78ms/step - loss: 1.5796 - accuracy: 0.4227 - val_loss: 1.8513 - val_accuracy: 0.2873
Epoch 3/50
782/782 [==============================] - 58s 75ms/step - loss: 1.4729 - accuracy: 0.4608 - val_loss: 1.8206 - val_accuracy: 0.3085
Epoch 4/50
268/782 [=========>....................] - ETA: 37s - loss: 1.4252 - accuracy: 0.4765

KeyboardInterrupt: 

In [ ]:
# Plot the training and validation loss on separate graphs
plt.subplot()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

In [ ]:
# Plot the training and validation accuracy on separate graphs
plt.subplot()
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.show()